In [1]:
data_folder = 'dmacp_data'

In [2]:
data_folder

'dmacp_data'

In [40]:
import json
import spacy
import pandas as pd

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# compressed content files
contributions_file = '/Users/francescamorini/projects/dmacp/dmacp_data/contributions.json'
field_notes_file = '/Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json'
projects_file = '/Users/francescamorini/projects/dmacp/dmacp_data/projects.json'
taxonomy_file = '/Users/francescamorini/projects/dmacp/dmacp_data/taxonomy_list.json'

# Some helper functions

def get_merged_contribution_text(contribution_data):
    texts = [contribution_data['abstract']]
    for entry in contribution_data['content']:
        texts.append(entry['text'])
    return ' '.join(texts)

def get_merged_field_note_text(field_note_data):
    return field_note_data['text']

def get_merged_project_text(project_data):
    texts = [project_data['abstract']]
    for entry in project_data['content']:
        texts.append(entry['text'])
    return ' '.join(texts)

In [41]:
import os.path
taxonomyLabels = []
with open(taxonomy_file) as taxonomy:
    taxonomy_list = json.load(taxonomy)
    for el in taxonomy_list:
        taxonomyLabels.append(el)

print(taxonomyLabels)

['methods', 'keywords', 'folksonomy']


In [42]:
#Helper function to check taxonomy content for each entry
def labelMethod(entryValue, currentTaxonomy):
    with open(taxonomy_file) as taxonomy:
        taxonomy_list = json.load(taxonomy)
        for taxonomy in taxonomy_list[currentTaxonomy]:

            if taxonomy['term_id'] == entryValue:
                return taxonomy['name']

In [85]:
allFiles = [contributions_file, field_notes_file, projects_file]
typeLabel = ['contribution', 'fieldnote', 'project']
temporalLabels = ['DATE', 'TIME']

index = 0

timedMethods = []
timedKeywords = []
timedFolksonomies = []
datasets = [timedMethods, timedKeywords, timedFolksonomies]

#Initial loops through types of taxonomy types
for taxonomy in taxonomyLabels:

    print('started', taxonomy)
    fileIndex = 0
    #Opening files
    for file in allFiles:
        with open(file) as f:

            entries = json.load(f)
            print('open', file)

              #Loop through entries
            for entry in entries:

                # print(entries[entry])
                single_entry = entries[entry]
                id = single_entry['id']
                metatime = single_entry['meta_datetime']
                if 'title' in single_entry:
                    title = single_entry['title']
                else:
                    title = typeLabel[fileIndex] + ' with no title'
                    
                for tax in single_entry[taxonomy]:
                    temporalEntities = []
                    taxonomy_id = tax
                    taxonomy_name = labelMethod(tax, taxonomy)

                
                datasets[index].append([title, id, metatime, taxonomy_name, typeLabel[fileIndex], taxonomy])
            fileIndex = fileIndex + 1
    index = index + 1
print('done', taxonomy)

started methods
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
started keywords
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
started folksonomy
open /Users/francescamorini/projects/dmacp/dmacp_data/contributions.json
open /Users/francescamorini/projects/dmacp/dmacp_data/field_notes.json
open /Users/francescamorini/projects/dmacp/dmacp_data/projects.json
done folksonomy


In [90]:
methods = pd.DataFrame(datasets[0], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy'])
keywords = pd.DataFrame(datasets[1], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy'])
folksonomy = pd.DataFrame(datasets[2], columns=['title','id', 'time', 'taxonomy_name', 'type_of_entry', 'type_of_taxonomy'])

In [79]:
keywords['id'].nunique()

655

In [91]:
methods.sort_values('title')

,title,id,time,taxonomy_name,type_of_entry,type_of_taxonomy
52,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Conversation,contribution,methods
3,30 Days On/30 Days Off with Paul Perkinson,20478,2020-02-24T14:23:37,Conversation,contribution,methods
4,A Brief History of Geoengineering,22468,2014-11-23T15:13:02,Film,contribution,methods
5,A Caribbean Taste of Technology: Creolization ...,28250,2017-04-15T14:21:22,Sound,contribution,methods
6,A Curriculum for the Anthropocene,26412,2016-04-01T10:01:53,Sound,contribution,methods
...,...,...,...,...,...,...
388,fieldnote with no title,14003,2019-10-18T15:23:23,Sensing,fieldnote,methods
387,fieldnote with no title,13917,2019-10-16T17:33:16,Sensing,fieldnote,methods
396,fieldnote with no title,14144,2019-10-23T09:15:48,Storytelling,fieldnote,methods
327,fieldnote with no title,11402,2019-09-18T20:37:19,Film,fieldnote,methods


In [92]:
keywords.sort_values('title')

,title,id,time,taxonomy_name,type_of_entry,type_of_taxonomy
52,#BlackBoyJoy On the River with Eugene B. Redmond,20440,2020-02-24T13:46:45,Water,contribution,keywords
3,30 Days On/30 Days Off with Paul Perkinson,20478,2020-02-24T14:23:37,Transportation,contribution,keywords
4,A Brief History of Geoengineering,22468,2014-11-23T15:13:02,Geo-engineering,contribution,keywords
5,A Caribbean Taste of Technology: Creolization ...,28250,2017-04-15T14:21:22,Technosphere,contribution,keywords
6,A Curriculum for the Anthropocene,26412,2016-04-01T10:01:53,Deep time,contribution,keywords
...,...,...,...,...,...,...
388,fieldnote with no title,14003,2019-10-18T15:23:23,Policy,fieldnote,keywords
387,fieldnote with no title,13917,2019-10-16T17:33:16,Policy,fieldnote,keywords
396,fieldnote with no title,14144,2019-10-23T09:15:48,Slavery,fieldnote,keywords
327,fieldnote with no title,11402,2019-09-18T20:37:19,Landscape,fieldnote,keywords
